In [3]:
!pip install fastexcel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.1 MB/s eta 0:00:00


In [35]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_business_data(file_path):
    """
    Analyzes business data using Polars and provides insights.

    Args:
        file_path: Path to the CSV file containing the business data.
    """
    try:
        df = pl.read_excel(file_path)
    except pl.exceptions.FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return

    # Data Cleaning and Preprocessing
    df = df.with_columns(
        [
            pl.col("Rating_count").str.replace(",", "").cast(pl.Int64),  # Remove commas and convert to integer
            pl.col("Price_range").str.len_chars().alias("Price_level"), # Convert price range to a numerical level
            pl.col("Tags").str.to_lowercase().str.split(",").alias("Tags_list"), # Convert tags to lowercase and split into list
        ]
    )

    # Exploratory Data Analysis (EDA)

    # Rating Distribution
    plt.figure(figsize=(8, 6))
    sns.histplot(df["Rating"].to_numpy(), kde=True, bins=20)
    plt.title("Distribution of Ratings")
    plt.xlabel("Rating")
    plt.ylabel("Frequency")
    plt.savefig("rating_distribution.png")
    plt.close() # Close the plot to avoid display in non-interactive environments

    # Rating vs. Rating Count
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x="Rating", y="Rating_count", data=df.to_pandas()) # Convert to pandas for seaborn scatterplot
    plt.title("Rating vs. Rating Count")
    plt.xlabel("Rating")
    plt.ylabel("Rating Count")
    plt.savefig("rating_vs_count.png")
    plt.close()


    # Top Categories
    top_categories = df.group_by("Category").count().sort("count", descending=True).limit(10)
    print("\nTop 10 Categories:\n", top_categories)

    # Average Rating by Category
    avg_rating_by_category = df.group_by("Category").agg(pl.col("Rating").mean().alias("Average_Rating")).sort("Average_Rating", descending=True)
    print("\nAverage Rating by Category:\n", avg_rating_by_category)


    # Tag Analysis (Example: Top 10 tags)

    all_tags = df.explode("Tags_list")["Tags_list"].drop_nulls()

    top_tags = all_tags.value_counts().sort("count", descending=True).limit(10)
    print("\nTop 10 Tags:\n", top_tags)

    # Example: Average rating for businesses with a specific tag (e.g., "tapas")
    if "tapas" in all_tags.unique():
        avg_rating_tapas = df.filter(pl.col("Tags_list").list.contains("tapas"))["Rating"].mean()
        print(f"\nAverage rating for businesses with 'tapas' tag: {avg_rating_tapas}")
    else:
        print("\nNo business with 'tapas' tag")

    # Price Level Analysis
    avg_rating_by_price = df.group_by("Price_level").agg(pl.col("Rating").mean().alias("Average_Rating")).sort("Price_level")
    print("\nAverage Rating by Price Level:\n", avg_rating_by_price)


if __name__ == "__main__":
    file_path = "Google Maps Listing and Details Page Scraper_tapas_madrid.xlsx"
    analyze_business_data(file_path)


Top 10 Categories:
 shape: (10, 2)
┌────────────────────────┬───────┐
│ Category               ┆ count │
│ ---                    ┆ ---   │
│ str                    ┆ u32   │
╞════════════════════════╪═══════╡
│ Tapas bar              ┆ 45    │
│ Restaurant             ┆ 9     │
│ Tapas restaurant       ┆ 8     │
│ Bar                    ┆ 6     │
│ Tavern                 ┆ 6     │
│ Spanish restaurant     ┆ 5     │
│ Bistro                 ┆ 4     │
│ Cocktail bar           ┆ 2     │
│ Bar & grill            ┆ 1     │
│ Traditional restaurant ┆ 1     │
└────────────────────────┴───────┘

Average Rating by Category:
 shape: (12, 2)
┌───────────────────────┬────────────────┐
│ Category              ┆ Average_Rating │
│ ---                   ┆ ---            │
│ str                   ┆ f64            │
╞═══════════════════════╪════════════════╡
│ Cocktail bar          ┆ 4.65           │
│ Gourmet grocery store ┆ 4.5            │
│ Spanish restaurant    ┆ 4.44           │
│ Bar          

<ipython-input-35-d74d6faf70b1>:49: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  top_categories = df.group_by("Category").count().sort("count", descending=True).limit(10)
